In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipynb.fs.full.chain_expander as chain
import refinitiv.data as rd
from datetime import datetime, timedelta
from dateutil.relativedelta import *
import calendar
from pandas.tseries.offsets import BMonthEnd
import warnings
warnings.filterwarnings('ignore')
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f4834c60820 {name='codebook'}>

In [36]:
def myfxaddperiod(t, dts):
    res_ = []
    res = {}
    for cd in dts:
        cd_ = cd
        for t_ in t:
            t_ = list(t_)
            for j in range(int(len(t_)/2)):
                n_ = t_[j*int(len(t_)/2)]
                p_ = t_[j*int(len(t_)/2)-1]
                if p_ == 'D':
                    cd_ += relativedelta(day=int(n_))
                elif p_ == 'M':
                    cd_ += relativedelta(months=int(n_))
                elif p_ == 'Y':
                    cd_ += relativedelta(years=int(n_))
            res_.append(cd_)
        res[cd] = res_
        res_ = []
    return res


def get_period(d):
    # 2020-01-31 vs 2019-05-03
    tod = datetime.today()
    yy = d.year - tod.year  # 1
    mm = d.month - tod.month  # -4
    _mm = yy * 12 + mm  # 8
    if _mm < 12:  # yes
        yy -= 1  # 0
        mm = _mm
    else:
        mm = abs(mm)
    dd = abs(d.day - tod.day)
    if dd > 15:
        mm += 1
        if mm > 12:
            mm = 0
            yy += 1
    yy = '{}Y'.format(yy) if yy > 0 else ''
    mm = '{}M'.format(mm) if mm > 0 else ''
    res = '{}{}'.format(yy, mm)
    return res


def get_expiry(d, delta):
    dd = 1
    mm = d.month + delta
    dy = 0
    while mm > 12:
        mm -= 12
        dy += 1
    yyyy = d.year + dy
    expiry = last_workday(datetime(year=yyyy, month=mm, day=dd))
    return expiry


def today():
    today_date = datetime.strptime(
        datetime.today().strftime('%Y-%m-%d'), '%Y-%m-%d')
    return today_date


def last_workday(d):
    last = BMonthEnd()
    return last.rollforward(d)


def str_to_date(str_val):
    return datetime.strptime(str_val, '%Y-%m-%d')


def date_to_str(d):
    dd = '%02d' % d.day
    mm = '%02d' % d.month
    yy = str(d.year)
    return '{}-{}-{}'.format(yy, mm, dd)


def get_curve(chain_ric):
    get_data = chain.get_data
    meta = get_data(chain_ric, '', 'futures')
    return meta


def gen_fut_crvs(df, d, cd):
    crv_dict = {}
    calc_dates = cd
    start_dt = min(calc_dates)
    temp_rics = df['ric'].tolist()
    root_ric = temp_rics[0][:-2]
    cont_rics = list(map(lambda i: '{}c{}'.format(
        root_ric, i + 1), df.index.values.tolist()))
    for i, t in enumerate(d):
        px = rd.get_history(cont_rics, 'SETTLE', start=str(pd.to_datetime(
            t) - timedelta(days=1)), end=str(t), interval='daily').dropna(axis=1)
        _xp = [last_workday(calc_dates[i])]
        for idx in range(1, len(px.columns)):
            _xp.append(get_expiry(_xp[idx-1], 1))
        crv_dict[t] = [_xp, px.values.tolist()[0]]
    return crv_dict


def create_tnr_dict(df):
    df['tenor'] = ''
    # change to iterrows()
    for i in range(len(df)):
        if df.EXPIR_DATE[i] == None:
            s = df['Instrument Description'][i]
            _s = s.split(' ')
            _l = len(_s)
            for _i, w in enumerate(_s):
                if w.lower() == 'month' or w.lower() == 'months' or w.lower() == 'mo' or w.lower() == 'mth' or w.lower() == 'mnth' or w.lower() == 'mths' or w.lower() == 'mnths':
                    c = ''
                    if _i == _l:
                        c = _s[_i-1]
                    else:
                        try:
                            try:
                                c = int(_s[_i-1])
                            except:
                                c = int(_s[_i + 1])
                        except:
                            0
                    if type(c) == int:
                        df.EXPIR_DATE[i] = get_expiry(
                            today(), c).strftime('%Y-%m-%d')
                        break
        try:
            df['tenor'][i] = get_period(df.EXPIR_DATE[i])

        except:
            0
    df = df.sort_values(by=['EXPIR_DATE'])
    df = df.reset_index(drop=True)
    return df


def get_data(spot, curve, dates=[date_to_str(today())], width=15, height=7, reduce_crv=None, interval='daily', count=365, num_contracts=24, save_image=False):
    calc_dates = list(map(lambda t: str_to_date(t), dates))
    if type(curve) == str:
        df = get_curve(curve)
        if num_contracts is not None:
            df = df.iloc[:num_contracts, :]
        crv_dict = gen_fut_crvs(df, dates, calc_dates)
    else:
        if type(curve) == list:
            _df = rd.get_data(
                curve, ['EXPIR_DATE', 'CF_NAME'])
            _df = create_tnr_dict(_df)

            curve = {}
            for i in range(len(_df)):
                curve[str(_df['tenor'][i])] = _df['Instrument'][i]

        if type(curve) == dict:
            p = []
            custom_rics = []
            for k in curve.keys():
                if k != 'SPOT':
                    p.append(k)
                    custom_rics.append(curve[k])
            crv_x = myfxaddperiod(p, calc_dates)
            crv_dict = {}
            for i, t in enumerate(dates):
                px = rd.get_history(custom_rics, 'SETTLE', start=str(pd.to_datetime(
                    t) - timedelta(days=1)), end=str(t), interval='daily').dropna(axis=1)
                crv_dict[t] = [crv_x[calc_dates[i]], px.values.tolist()[0]]

    if spot != '' and spot != None:
        end = today()
        spot_hist = rd.get_history(spot, 'TRDPRC_1', end=str(end), start=str(datetime(
            year=end.year - 1, month=end.month, day=end.day)), interval=interval).dropna(axis=1)

    plt.figure(figsize=[width, height])

    clrs = ['#6978F7', '#A325E9', '#96E05D', '#4A7FB9',
            '#E75A2D', '#FBE55A', '#8C8C8D', '#5A54F6']
    plt.style.use('dark_background')
    plt.plot(spot_hist.index, spot_hist['TRDPRC_1'].tolist(), color='#6978F7')

    plt.grid(color='lightgray', linestyle='--', linewidth=0.5)

    for i, j in enumerate(crv_dict.keys()):
        if reduce_crv is not None:
            u = int(len(crv_dict[j][0])/reduce_crv)
            x = crv_dict[j][0][:u]
            y = crv_dict[j][1][:u]
        else:
            x = crv_dict[j][0]
            y = crv_dict[j][1]
        plt.plot(x, y, label=j, color=clrs[i + 1],
                 marker='o', linewidth=0.5, markersize=4)

    l, h = plt.ylim()
    if h >= 0 and l <= 0:
        plt.axhline(y=0, linewidth=1.5, color='black')
    plt.legend(loc='upper right')
    plt.tight_layout()

    if save_image:
        plt.savefig('term_structure.png')

    return

In [14]:
rd.close_session()